In [ ]:
!pip install ipykernel transformer_lens plotly 

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import transformer_lens
import torch
# import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
import numpy as np

from jaxtyping import Float
from functools import partial
import numpy as np

/nlp/projekty/aver/xspiege1/algorithmic_reasoning/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Convert HF Transformers weights to HookedTransformer weights

In [3]:
"""
from transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_config(config)
model.load_state_dict(torch.load("checkpoints/llama_instruct_addition_finetuned_shorted_OOD.weights", weights_only=True))
hooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
new_state_dict = convert_llama_weights(model, hooked_config)
torch.save(new_state_dict, "checkpoints/llama_instruct_addition_finetuned_hooked.weights")
"""


'\nfrom transformer_lens.pretrained.weight_conversions.llama import convert_llama_weights\nfrom transformers import AutoConfig, AutoModelForCausalLM\n\nconfig = AutoConfig.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")\nmodel = AutoModelForCausalLM.from_config(config)\nmodel.load_state_dict(torch.load("checkpoints/llama_instruct_addition_finetuned_shorted_OOD.weights", weights_only=True))\nhooked_config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")\nnew_state_dict = convert_llama_weights(model, hooked_config)\ntorch.save(new_state_dict, "checkpoints/llama_instruct_addition_finetuned_hooked.weights")\n'

In [4]:
config = transformer_lens.loading.get_pretrained_model_config("meta-llama/Llama-3.2-1B-Instruct")
model = transformer_lens.HookedTransformer(config)
model.load_and_process_state_dict(torch.load("checkpoints/llama_instruct_addition_finetuned_hooked.weights", weights_only=True))
#config = transformer_lens.loading.get_pretrained_model_config("Qwen/Qwen2.5-1.5B-Instruct")
#model = transformer_lens.HookedTransformer(config)
#model.load_and_process_state_dict(torch.load("checkpoints/Qwen2.5-1.5B-Instruct_value_assign_finetuned_hooked.weights", weights_only=True))


In [5]:
print(model)

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-15): 16 x TransformerBlock(
      (ln1): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out)

In [6]:
charset_from = ""
for id in range(0, 50000):
    token = model.tokenizer.decode(id, skip_special_tokens=True)
    if len(token) > 1 or token == "�":
        continue
    decode_id = model.tokenizer.encode(token, add_special_tokens=False)
    if len(decode_id) == 1 and decode_id[0] == id:
        charset_from += token

In [7]:
charset_from[:105]

'!"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n'

In [8]:
import sys
sys.path.insert(0, "..")
sys.path.insert(0, "../generators")
sys.path.insert(0, "generators")


from generators.long_addition import LongAdditionGenerator
import transformers
import itertools


data_generator = LongAdditionGenerator(tokenizer=model.tokenizer,
                                       seed=0,
                                       use_few_shot=True, 
                                       use_instruction=True, 
                                       apply_chat_template=True,
                                       num_of_nums=(2, 2),
                                       length=(4, 4)
                                       )
samples = itertools.islice(data_generator.generate_samples(), 30)

In [9]:
samples = list(samples)
texts = [sample[0] for sample in samples]
input_ids = [sample[1] for sample in samples]
target_ids = [sample[2] for sample in samples]
attn_labels = [sample[3] for sample in samples]

In [10]:
texts[0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 18 May 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n\nYou are an expert in solving arithmetic tasks. You will be tasked with computing the sum of input numbers.\nThe number are all represented with their least significant digit first.\nSum these number using the standard addition algorithm. When predicting each digit of the result, always sum the corresponding digits of each number and remember the carry.\nDo not generate any words. You can only generate number and nothing else.\n\nHere are some examples:\n1+1=2\n1240+4335+3440=8916\n999+999=8991\nUse them to solve the following task:\n36610+67390=<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n93011<|eot_id|>'

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import random
import plotly.graph_objects as go


def visualize_attention_heads(attention_matrix, tokens=None, title="Attention Heatmap", save_path=None, heads=None, length=None, attn_labels=None):
    """
    Visualize a multi-head attention matrix as heatmaps for each head.

    Parameters:
        attention_matrix (torch.Tensor): Tensor of shape (1, num_heads, seq_len, seq_len).
        tokens (list, optional): List of tokens corresponding to the rows/columns of the matrix.
        title (str): Title of the heatmap.
        save_path (str, optional): Path to save the heatmap images. If None, the heatmaps are not saved.
        heads (list, optional): List of head indices to visualize.
        length (int, optional): Number of tokens to display from the lower-right corner.
"""
    scores = attention_matrix.squeeze(0).cpu().detach().numpy()  # Shape: (num_heads, seq_len, seq_len)
    num_heads = scores.shape[0]
    seq_len = scores.shape[1]

    for head in range(num_heads):
        if heads is not None and head not in heads:
            continue
        head_scores = scores[head]  # Shape: (seq_len, seq_len)

        # Apply filtering for the lower-right "length x length" square
        if length is not None and length < seq_len:
            head_scores = head_scores[-length:, -length:]
            if tokens:
                tokens = tokens[-length:]
            if attn_labels is not None:
                attn_labels = attn_labels[-length:, -length:]

        # Normalize scores for better visualization
        min_score = head_scores.min()
        max_score = head_scores.max()
        normalized = (head_scores - min_score) / (max_score - min_score)  # Normalize to [0, 1]
        print("fig")
        # Create the heatmap
        fig = go.Figure(
            data=go.Heatmap(
                z=normalized,
                x=tokens if tokens else list(range(len(normalized))),
                y=tokens if tokens else list(range(len(normalized))),
                colorscale="Viridis",
                colorbar=dict(title="Attention Score"),
            )
        )
        
        # Add title and layout
        fig.update_layout(
            title=f"{title} - Head {head + 1}",
            xaxis=dict(title="Tokens", tickangle=45),
            yaxis=dict(title="Tokens"),
        )
        print("attn")
        # Highlight attention label regions if provided
        if attn_labels is not None:
            for i in range(len(attn_labels)):
                for j in range(len(attn_labels)):
                    if attn_labels[i, j] == 1:
                        fig.add_shape(
                            type="rect",
                            x0=j - 0.5,
                            x1=j + 0.5,
                            y0=i - 0.5,
                            y1=i + 0.5,
                            line=dict(color="red", width=2),
                        )
        print("finish")
        # Save the plot if a save path is provided
        if save_path:
            fig.write_image(f"{save_path}_head_{head + 1}.png")

        # Show the plot
        #fig.show()

In [ ]:
best_heads = {}

def compute_scores(logits, target):
    ignore_index = -100  # Optional: index to ignore in loss/accuracy computation
    # Compute loss
    loss = torch.nn.functional.cross_entropy(
    logits.view(-1, logits.size(-1)),  # Flatten logits to (batch_size * seq_len, vocab_size)
    target.view(-1),               # Flatten target_ids to (batch_size * seq_len)
    ignore_index=ignore_index          # Ignore padding tokens if applicable
)

    # Compute accuracy
    predictions = torch.argmax(logits, dim=-1)  # Get predicted token indices
    correct = (predictions == target) & (target != ignore_index)  # Ignore padding tokens
    accuracy = correct.sum().item() / (target != ignore_index).sum().item()

    print(f"Loss: {loss.item()}, Accuracy: {accuracy:.2%}")

    errors = (predictions != target) & (target != ignore_index)  # Identify error positions
    errors_np = errors.cpu().numpy()  # Convert to NumPy for visualization
    #print("Errors:", errors_np)

    return loss.item(), accuracy

def fix_attn_pattern_hook(
    value: Float[torch.Tensor, "batch pos head_index d_head"],
    hook: HookPoint,
    attn_labels_1_idx,
    heads_to_reinforce,
    attn_labels,
    keep_last_n_tokens,
    threshold=3e-3
) -> Float[torch.Tensor, "batch pos head_index d_head"]:
    batch, heads, seq_len, _ = value.shape 
    batch, heads, seq_len, _ = value.shape
    layer = int(hook.name.split(".")[1])  # Extract layer number from hook name

    if layer not in heads_to_reinforce.keys():
        return value

    # Create a mask from attn_labels_1_idx
    mask = torch.zeros((seq_len, seq_len), dtype=torch.bool, device=value.device)
    indices_tensor = torch.tensor(list(attn_labels_1_idx), dtype=torch.long, device=value.device)
    mask[indices_tensor[:, 0], indices_tensor[:, 1]] = True

    # Apply the mask and threshold for each head
    for h in range(heads):
        if h not in heads_to_reinforce[layer]:
            continue

        # Get the attention scores for the current head
        head_scores = value[0, h]

        # Apply the mask and threshold
        masked_scores = head_scores[mask]
        above_threshold = masked_scores >= threshold

        # Update the attention scores for positions that meet the condition
        head_scores[mask] += above_threshold.float()
        visualize_attention_heads(value, None, title=f"Example Attention Heatmap Layer {layer}", heads=[h], length=keep_last_n_tokens, attn_labels=None, save_path="vis2/add")
        return value

device = "cuda"

is_attn_scores = lambda name: name.endswith("hook_pattern")
ablated = []
for idx in range(0, 30):



    
    indices =  np.argwhere(attn_labels[idx].cpu().numpy() == 1)
    attn_labels_1_idx = set(map(tuple, indices))
    

    model.to(device)
    input_ids[idx] = input_ids[idx].to(device)
    target_ids[idx] = target_ids[idx].to(device)
    attn_labels[idx] = attn_labels[idx].to(device)
                

    heads_to_reinforce_x=[
        (0, range(32)),
        (1, range(32)),
        (2, range(32)),
        (3, range(32)),
        (4, range(32)),
        (5, range(32)),
        (6, range(32)),
        (7, range(32)),
        (8, range(32)),
        (9, range(32)),
        (10, range(32)),
        (11, range(32)),
        (12, range(32)),
        (13, range(32)),
        (14, range(32)),
        (15, range(32)),
    ]
    threshold = 0.02
    for threshold in np.linspace(0.002, 0.002, 1):
        cols_to_keep = ~np.all(attn_labels[idx].cpu().numpy() == -100, axis=0)
        keep_last_n_tokens = 400
        heads_x = dict(heads_to_reinforce_x)
        print(threshold)
        logits = model.run_with_hooks(input_ids[idx], return_type="logits",
                                        fwd_hooks=[
                                            (is_attn_scores, partial(fix_attn_pattern_hook, attn_labels_1_idx=attn_labels_1_idx, heads_to_reinforce=heads_x, attn_labels=attn_labels[idx], keep_last_n_tokens = keep_last_n_tokens, threshold=threshold))
                                        ])
        loss, acc = compute_scores(logits, target_ids[idx])
        ablated.append((heads_x, loss, acc))
        del logits
        torch.cuda.empty_cache()
    

Moving model to device:  cuda
0.002
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
Loss: 0.6983170509338379, Accuracy: 66.67%
Moving model to device:  cuda
0.002
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
Loss: 0.7976048588752747, Accuracy: 66.67%
Moving model to device:  cuda
0.002
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
fig
attn
finish
Loss: 0.7739169001579285, Accuracy: 66

In [ ]:
len(input_ids[0])

204

In [ ]:
sums = 0
for item in ablated:
    sums += item[2]
print(sums/len(ablated))
print(sums)

In [ ]:
for item in sorted(best_heads.items(), key=lambda item: item[1], reverse=True):
    print(item[0][0], item[0][1], item[1].item())